In [ ]:
import numpy as np
from dadk.QUBOSolverCPU import *
from dadk.BinPol import *
from datetime import datetime
import pandas as pd

# Input Data

In [ ]:
# Input data 

# Assets
asset_quantity = pd.read_csv("data/sample_asset_quantity.csv", header=None)[0].values.tolist()
asset_value = pd.read_csv("data/sample_asset_value.csv", header=None)[0].values.tolist()
asset_tiers = pd.read_csv("data/sample_asset_tiers.csv", header=None)[0].values.tolist()

n_assets = len(asset_quantity)

# Accounts 
account_exposure = pd.read_csv("data/sample_account_exposure.csv", header=None)[0].values.tolist()
account_duration = pd.read_csv("data/sample_account_duration.csv", header=None)[0].values.tolist()

n_accounts = len(account_exposure)

# Haircuts
haircuts = (pd.read_csv("data/sample_haircuts.csv", header=None).values.tolist())

#Single Limits
#single_limits = (pd.read_csv("data/sample_single_limits.csv", header=None).values.tolist())

# Updated Tier list 
cost_factor_matrix = np.zeros(shape=(n_assets, n_accounts))

for i in range(n_assets):
    for j in range(n_accounts):
        cost_factor_matrix[i,j] = abs(account_duration[j] - asset_tiers[i])

 ## Penalty Terms


In [ ]:
# Penalty terms
lambda_cost_fn = 1e4
lambda_consistency1, lambda_consistency2 = 1, 1
lambda_exposure1, lambda_exposure2 = 1, 1e6

#lambda_single_lim1, lambda_single_lim2 = 1,1

# Constructing the QUBO

In [ ]:
# n-bit binary variables
n_binary = 7
binary_coeff = []
max_binary_value = (2**n_binary) - 1

for b in range(1,n_binary+1):
    aux_term = (2**(b-1) * 1) / max_binary_value
    binary_coeff.append(aux_term)

binary_variables = VarShapeSet(BitArrayShape(name='bin_vars', shape=(n_assets, n_accounts, n_binary), axis_names=['Assets', 'Accounts', 'Binary']))


In [ ]:
# Constructing QUBO equation

# Objective function
cost_function = BinPol(binary_variables)
for i in range(n_assets):
    for j in range(n_accounts):
        for k in range(n_binary):
            cost_function.add_term(cost_factor_matrix[i][j]*binary_coeff[k],(("bin_vars", i, j, k),))

In [ ]:
#consistency requirement
con1 = BinPol(binary_variables)
con2 = BinPol(binary_variables)

con_constraint = BinPol(binary_variables)
for i in range(n_assets):
    aux_term = BinPol(binary_variables)
    for j in range(n_accounts):
        for k in range(n_binary):
            aux_term.add_term(binary_coeff[k],(("bin_vars", i, j, k),))
        aux_term.add_term(-1,())
    con1.add(aux_term)
    aux_term.power(2)
    con2.add(aux_term)

con1.multiply_scalar(lambda_consistency1)
con2.multiply_scalar(lambda_consistency2)

con_constraint = con1 + con2


In [ ]:
# Exposure requirements
exposure1  = BinPol(binary_variables)
exposure2 = BinPol(binary_variables)


for j in range(n_accounts):
    aux_term = BinPol(binary_variables)
    for i in range(n_assets):
        for k in range(n_binary):
            aux_term.add_term(binary_coeff[k]*asset_quantity[i]*asset_value[i]*haircuts[i][j]/100,(("bin_vars", i, j, k),))
    aux_term.add_term(-account_exposure[j],())
    exposure1.add(aux_term)
    aux_term.power(2)
    exposure2.add(aux_term)

exposure1.multiply_scalar(lambda_exposure1/(np.mean(np.array(asset_quantity)*np.array(asset_value))))
exposure2.multiply_scalar(lambda_exposure2/(np.mean(np.array(asset_quantity)*np.array(asset_value))**2))

exposure_requirements = -exposure1 + exposure2


In [ ]:
# #single_limits
# single1 = BinPol(var_shape_set)
# single2  = BinPol(var_shape_set)


# for i in range(n_assets):
#     for j in range(n_accounts):
#         step = BinPol(var_shape_set)
#         for k in range(nbit):
#             step.add_term(binary_coeff[k]*asset_quantity[i],(("bin_vars", i, j, k),))
#         step.add_term(-single_limits[i][j],())
#         single1.add(step)
#     step.power(2)
# #         print(step)
#     single2.add(step)
# #     print(step)
# #     print('--')
        
# single1.multiply_scalar(lambda_single_lim1)
# single2.multiply_scalar(lambda_single_lim2)

# single_limit = single1 + single2


In [ ]:
# Final QUBO
final_equation = (cost_function*lambda_cost_fn + con_constraint + exposure_requirements) #+ single_limit
final_equation *= 1e4 #scaling factor to avoid rounding of smaller values
#To include single limits into the qubo uncomment all lines refering to them and the additional validation below

# Run Solver

In [ ]:
solver = QUBOSolverCPU(
   
    number_iterations    = 50000,               # total number of itrations per run
    number_runs          = 4,                 # number of stochastically independant runs
    temperature_start    = 5000,               # start temperature for annealing as float value
    temperature_end      = 10,                 # end temperature for annealing as float value 
    temperature_mode     = 0,                         # 0: reduce temperature by factor (1-temperature_decay) every temperature_interval steps
                                                      # 1: reduce temperature by factor (1-temperature_decay*temperature) every temperature_interval steps
                                                      # 2: reduce temperature by factor (1-temperature_decay*temperature^2) every temperature_interval steps
    temperature_decay    = 0.0095,             # see temperature_mode 0
    temperature_interval = 1,                  # see temperature_mode 0
    offset_increase_rate = 5000.0,             # increase of dynamic offset when no bit selected, set to 0.0 to switch off dynamic offset
    graphics             = True                # create data for graphics output
)


In [ ]:
# Run solver
start=datetime.now()
solution_list = solver.minimize(final_equation)

# Get results
solution = solution_list.get_minimum_energy_solution()
configuration = solution.configuration
my_bit_array = solution.extract_bit_array("bin_vars")

# Verify Results

In [ ]:
bitstring = ''

exposure_solution = np.zeros(shape = (n_accounts))
allocation_solution = np.zeros(shape = (n_assets))

print("=====Each Individual Allocation Percentage=====")

for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(n_binary):
            bit += str(my_bit_array[i][j][k])
            decimal +=(2**k)*my_bit_array[i][j][k]
        asset += decimal*(100/max_binary_value)
        print(f"Assign {decimal*(100/max_binary_value):0.2f}% of asset {i+1} to account {j+1}")
        exposure_solution[j] += (decimal /max_binary_value) * asset_value[i] * asset_quantity[i]*haircuts[i][j]/100
    allocation_solution[i] = asset


In [ ]:
print("====Validation for Consistency====")
for i in range(n_assets):
    print(f"{allocation_solution[i]:0.2f}% of asset {i+1} posted")

In [ ]:
print("=====Validation for Exposure======")
for i in range(n_accounts):
    print(f"${exposure_solution[i]:,} of collateral posted, ${account_exposure[i],:} required")


In [ ]:
# #print("===Validation of Single limits =====")
# for i in range(n_assets):
#     for j in range(n_accounts):
#         decimal = 0
#         for k in range(n_binary):
#             decimal +=(2**k)*my_bit_array[i][j][k]
#         allocation = (decimal/max_binary_value)*asset_quantity[i] 
#         print(f"Amount allocated: {allocation:0.2f}. Single Limit: {single_limits[i][j]:0.2f}")
#         print(f"Constraint Satisfied?: {allocation <= single_limits[i][j]}")

In [ ]:
print(f"The objective function is: {cost_function.compute(configuration)}")
print("The optimal objective function is: 0.4299816571359823")

In [ ]:
print(datetime.now()-start)

# Export to .csv

In [ ]:
# Export to .csv

Q_value_balanced_py_qubo = np.zeros(shape=(n_assets,n_accounts))


for i in range(n_assets):
    asset = 0
    for j in range(n_accounts):
        bit = ''
        decimal = 0
        for k in range(n_binary):
            bit += str(my_bit_array[i][j][k])
            decimal +=binary_coeff[k]*my_bit_array[i][j][k]*max_binary_value
        asset += decimal*(100/max_binary_value)
        Q_value_balanced_py_qubo[i][j] = decimal/max_binary_value

#Uncomment the line below to save the Data to csv

# pd.DataFrame(Q_value_balanced_py_qubo).to_csv("Q_value_unbalanced_fujitsu_no_single_lim3.csv", header=None, index=None)